In [1]:
import operator as ops
import pathlib
from collections import Counter, defaultdict
from itertools import count

import numpy as np
import pandas as pd
import seaborn as sns
import torch
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

from arkanoid import Arkanoid
from dqn import DQNAgent
from nes_py.wrappers import JoypadSpace
from torchinfo import summary

ACTIONS = [["NOOP"], ["left"], ["right"], ["A"]]

env = JoypadSpace(Arkanoid(False), ACTIONS)

In [2]:
checkpoint_dir: pathlib.Path = pathlib.Path("checkpoints")
batch_size: int = 32
episodes: int = 1200
save_every: int = 10_000

agent = DQNAgent(env, batch_size=batch_size)

checkpoint_dir.mkdir(parents=True, exist_ok=True)
latest_dir = checkpoint_dir / "latest"
latest_dir.mkdir(exist_ok=True)

c = count()


def rgb2gray(rgb):
    return np.dot(rgb[..., :3], [0.2989, 0.5870, 0.1140])


summary(agent.policy_net)

Layer (type:depth-idx)                   Param #
DQN                                      --
├─DQN_TAB: 1-1                           --
│    └─Sequential: 2-1                   --
│    │    └─Linear: 3-1                  17,408
│    │    └─ReLU: 3-2                    --
│    │    └─Linear: 3-3                  262,400
│    │    └─ReLU: 3-4                    --
├─DQN_CNN: 1-2                           --
│    └─Sequential: 2-2                   --
│    │    └─Conv2d: 3-5                  2,080
│    │    └─MaxPool2d: 3-6               --
│    │    └─GELU: 3-7                    --
│    │    └─Conv2d: 3-8                  32,832
│    │    └─MaxPool2d: 3-9               --
│    │    └─GELU: 3-10                   --
│    │    └─Conv2d: 3-11                 36,928
│    │    └─MaxPool2d: 3-12              --
│    │    └─GELU: 3-13                   --
│    │    └─Flatten: 3-14                --
│    │    └─Linear: 3-15                 12,320
│    │    └─GELU: 3-16                   --
├─L

In [3]:
for episode in tqdm(range(1, episodes + 1), desc="Episode: ", position=0):
    screen, info = env.reset()
    episode_score = 0

    screen = torch.tensor(
        rgb2gray(env.crop_screen(screen)), dtype=torch.float32, device=agent.device,
    ).unsqueeze(0)
    info = torch.tensor(env.info_to_array(info), device=agent.device)

    for frame in tqdm(c, desc="Frame: ", position=1):
        action = agent.get_action(screen, info)
        next_screen, reward, done, next_info = env.step(action)

        next_screen = torch.tensor(
            rgb2gray(env.crop_screen(next_screen)),
            dtype=torch.float32,
            device=agent.device,
        ).unsqueeze(0)
        next_info = torch.tensor(env.info_to_array(next_info), device=agent.device)

        episode_score += reward
        agent.update(screen, info, action, reward, done, next_screen, next_info)

        screen = next_screen
        info = next_info

        if save_every is not None and frame % save_every == 0:
            checkpoint_dir_ = checkpoint_dir / f"{frame}"
            checkpoint_dir_.mkdir(exist_ok=True)
            torch.save(
                agent.policy_net.state_dict(), checkpoint_dir_ / "policy_net.pth",
            )
            torch.save(
                agent.policy_net.state_dict(), latest_dir / "policy_net.pth",
            )

        if done:
            losses = agent.loss[env.episode]
            agent.scores[env.episode] = episode_score
            print(
                f"Episode {episode}: final score={env.game['score']} total rewards={episode_score} mean loss = {torch.mean(torch.tensor(losses)):.4f}",
                flush=True,
            )
            break

Episode:   0%|          | 0/1200 [00:00<?, ?it/s]

Frame: : 0it [00:00, ?it/s]

Episode 1: final score=430 total rewards=380.0 mean loss = 0.8320


Frame: : 0it [00:00, ?it/s]

Episode 2: final score=160 total rewards=-370.0 mean loss = 0.8823


...

Frame: : 0it [00:00, ?it/s]

Episode 156: final score=240 total rewards=-80.0 mean loss = 0.8945


Frame: : 0it [00:00, ?it/s]

Exception ignored in: <function tqdm.__del__ at 0x7f15377ceb60>
Traceback (most recent call last):
  File "/home/nox/repos/arkanoid-gym/venv/lib/python3.11/site-packages/tqdm/std.py", line 1144, in __del__
    def __del__(self):

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
torch.save(agent.policy_net.state_dict(), checkpoint_dir / "policy_net.pth")
torch.save(agent.policy_net.state_dict(), latest_dir / "policy_net.pth")

In [ ]:
episode_durations = agent.durations
sns.scatterplot(x=episode_durations.keys(), y=episode_durations.values(), s=10)
plt.title("Duration per episode")
plt.ylabel("Frames")
plt.xlabel("Episode")
plt.show()

In [ ]:
episodes_scores = agent.scores
sns.scatterplot(
    x=np.arange(1, len(episodes_scores) + 1), y=list(episodes_scores.values()), s=10
)
plt.title("Score per episode")
plt.ylabel("score")
plt.xlabel("Episode")
plt.show()

In [ ]:
df = pd.DataFrame(agent.actions).T
df.rename(columns=lambda x: ACTIONS[x][0], inplace=True)
df = df.div(df.sum(axis=1), axis=0)

In [ ]:
sns.barplot(
    data=df.melt(ignore_index=False, value_name="prop", var_name="action")
    .reset_index()
    .rename(columns={"index": "episode"}),
    x="episode",
    y="prop",
    hue="action",
)